In [16]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]

In [4]:
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [5]:
def f(series):
    return series.shift(1).cumsum().fillna(0) / np.arange(len(series))


In [6]:
df["target_enc_content_id"] = df.groupby("content_id")["answered_correctly"].transform(f)

# content_id=3628(question)/content_id=27845(lecture)

In [45]:
def f(series, content_id):
    return ((series == content_id).cumsum() > 0)

In [26]:
content_id_included_user_id = df[df["content_id"] == 3628]["user_id"].drop_duplicates().values
w_df = df[df["user_id"].isin(content_id_included_user_id)].reset_index()

In [47]:
w_df.columns

Index(['index', 'timestamp', 'user_id', 'content_id', 'content_type_id',
       'task_container_id', 'user_answer', 'answered_correctly',
       'prior_question_elapsed_time', 'prior_question_had_explanation',
       'bundle_id', 'correct_answer', 'part', 'tags', 'tag', 'type_of',
       'target_enc_content_id', 'lectured', 'cumsum'],
      dtype='object')

In [59]:
w_df["lectured"] = w_df.groupby("user_id")["content_id"].transform(f, **{"content_id": 27845})
ww_df = w_df[w_df["content_id"] == 3628]
ww_df["cumsum"] = ww_df.groupby("user_id").cumcount()
ww_df.groupby(["lectured"])["answered_correctly"].describe()

/home/owner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,count,mean,std,min,25%,50%,75%,max
lectured,,,,,,,,
False,2295.0,0.524619,0.499502,0.0,0.0,1.0,1.0,1.0
True,622.0,0.765273,0.424169,0.0,1.0,1.0,1.0,1.0


count      mean       std  min   25%  50%  75%  max
lectured cumsum                                                      
False    0       2175.0  0.517701  0.499801  0.0  0.00  1.0  1.0  1.0
         1        103.0  0.631068  0.484875  0.0  0.00  1.0  1.0  1.0
         2         13.0  0.769231  0.438529  0.0  1.00  1.0  1.0  1.0
         3          4.0  0.750000  0.500000  0.0  0.75  1.0  1.0  1.0
True     0        163.0  0.595092  0.492387  0.0  0.00  1.0  1.0  1.0
         1        344.0  0.822674  0.382500  0.0  1.00  1.0  1.0  1.0
         2         85.0  0.835294  0.373116  0.0  1.00  1.0  1.0  1.0
         3         23.0  0.782609  0.421741  0.0  1.00  1.0  1.0  1.0
         4          5.0  1.000000  0.000000  1.0  1.00  1.0  1.0  1.0
         5          1.0  1.000000       NaN  1.0  1.00  1.0  1.0  1.0
         6          1.0  1.000000       NaN  1.0  1.00  1.0  1.0  1.0

In [50]:
w_df["lectured"] = w_df.groupby("user_id")["content_id"].transform(f, **{"content_id": 29680})
ww_df = w_df[w_df["content_id"] == 3628]
ww_df["cumsum"] = ww_df.groupby("user_id").cumcount()
ww_df.groupby(["lectured", "cumsum"])["answered_correctly"].describe()

/home/owner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


count      mean       std  min  25%  50%  75%  max
lectured cumsum                                                     
False    0       2227.0  0.522676  0.499598  0.0  0.0  1.0  1.0  1.0
         1        411.0  0.773723  0.418931  0.0  1.0  1.0  1.0  1.0
         2         84.0  0.821429  0.385293  0.0  1.0  1.0  1.0  1.0
         3         24.0  0.791667  0.414851  0.0  1.0  1.0  1.0  1.0
         4          4.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0
         5          1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0
         6          1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0
True     0        111.0  0.531532  0.501268  0.0  0.0  1.0  1.0  1.0
         1         36.0  0.833333  0.377964  0.0  1.0  1.0  1.0  1.0
         2         14.0  0.857143  0.363137  0.0  1.0  1.0  1.0  1.0
         3          3.0  0.666667  0.577350  0.0  0.5  1.0  1.0  1.0
         4          1.0  1.000000       NaN  1.0  1.0  1.0  1.0  1.0

,count,mean,std,min,25%,50%,75%,max
lectured,,,,,,,,
False,2788.0,0.576040,0.494273,0.0,0.0,1.0,1.0,1.0
True,129.0,0.573643,0.496475,0.0,0.0,1.0,1.0,1.0


In [60]:
w_df["lectured"] = w_df.groupby("user_id")["content_id"].transform(f, **{"content_id": 6808})
ww_df = w_df[w_df["content_id"] == 3628]
ww_df["cumsum"] = ww_df.groupby("user_id").cumcount()
ww_df.groupby(["lectured"])["answered_correctly"].describe()

/home/owner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,count,mean,std,min,25%,50%,75%,max
lectured,,,,,,,,
False,2606.0,0.567153,0.495565,0.0,0.0,1.0,1.0,1.0
True,311.0,0.649518,0.477890,0.0,0.0,1.0,1.0,1.0


In [61]:
w_df["lectured"] = w_df.groupby("user_id")["content_id"].transform(f, **{"content_id": 28921})
ww_df = w_df[w_df["content_id"] == 3628]
ww_df["cumsum"] = ww_df.groupby("user_id").cumcount()
ww_df.groupby(["lectured"])["answered_correctly"].describe()

/home/owner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,count,mean,std,min,25%,50%,75%,max
lectured,,,,,,,,
False,2788.0,0.576040,0.494273,0.0,0.0,1.0,1.0,1.0
True,129.0,0.573643,0.496475,0.0,0.0,1.0,1.0,1.0


In [53]:
df[df["content_type_id"] == 1].tail(10)

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,bundle_id,correct_answer,part,tags,tag,type_of,target_enc_content_id
196314,3567314671,2146672950,12294,1,1774,-1,NaN,NaN,False,-1,-1,5,NaN,27195,solving question,0.063776
196315,3567927428,2146672950,8708,1,1788,-1,NaN,NaN,False,-1,-1,5,NaN,12586,concept,0.000000
196316,3568313086,2146672950,28921,1,1792,-1,NaN,NaN,False,-1,-1,5,NaN,9680,concept,0.000000
196317,3614072597,2146672950,27022,1,1799,-1,NaN,NaN,False,-1,-1,5,NaN,31458,concept,0.000000
196318,3614218905,2146672950,814,1,1804,-1,NaN,NaN,False,-1,-1,5,NaN,7547,solving question,0.000000
196319,259919528,2146841500,6808,1,51,-1,NaN,NaN,False,-1,-1,2,NaN,21858,intention,0.233579
196320,789799549,2147056350,3153,1,76,-1,NaN,NaN,False,-1,-1,2,NaN,29260,intention,0.000000
196321,3362950,2147160110,16136,1,20,-1,NaN,NaN,False,-1,-1,2,NaN,17316,concept,0.000000
196322,2300304011,2147470770,89,1,168,-1,NaN,NaN,False,-1,-1,5,NaN,24584,concept,0.000000
196323,2300332267,2147470770,12472,1,170,-1,NaN,NaN,False,-1,-1,5,NaN,13950,concept,0.000000


In [56]:
df.groupby("content_type_id")["content_id"].nunique()

content_type_id
0    13509
1      408
Name: content_id, dtype: int64